In [10]:
import pyads
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import os


plc_loadcyc_name = 'Main.load_cycle'
plc_loadsucced_name = 'Main.FB_NN.flag_LoadWeights'
plc_readlen_name = 'Main.FB_NN.readlen'
plc_loadfile_name = 'Main.FB_NN.filePath'
plc_MeasureFinished_name = 'Main.MeasureFinished'

filetemplate = r'c:\Users\Administrator\Documents\git-projekte\keras2plc\experiment\loading_time\pseudo_weights_file\weights_[layers]_[neurons].dat'


num_layers = [6, 5, 4, 3]
num_neurons = [500, 400, 300, 200, 100, 10]
input_dim = 5
output_dim = 1

In [11]:
load_cyc = []
with pyads.Connection('127.0.0.1.1.1', pyads.PORT_TC3PLC1) as plc:
    plc.write_by_name(plc_loadcyc_name, 0, pyads.PLCTYPE_UINT)
    for i in num_layers:
        load_cyc.append([])
        for j in num_neurons:
            file = filetemplate.replace('[layers]',f'{i}').replace('[neurons]',f'{j}')
            plc.write_by_name(plc_loadfile_name, file,pyads.PLCTYPE_STRING)

            num_neurons_layers = []
            num_neurons_layers.extend((input_dim,*[j for k in range(i-2)],output_dim))

            num_lreal = 0
            for ind,num in enumerate(num_neurons_layers[:-1]):
                num_lreal += (num * num_neurons_layers[ind+1] + num_neurons_layers[ind+1])
            plc.write_by_name(plc_readlen_name, num_lreal,pyads.PLCTYPE_UINT)
            plc.write_by_name(plc_loadsucced_name, False,pyads.PLCTYPE_BOOL)
            plc.write_by_name(plc_MeasureFinished_name, False, pyads.PLCTYPE_BOOL)
            while(1):
                loadsucced = plc.read_by_name(plc_loadsucced_name, pyads.PLCTYPE_BOOL)
                if loadsucced:
                    break
            times_cyc = plc.read_by_name(plc_loadcyc_name, pyads.PLCTYPE_UINT)
            load_cyc[-1].append(times_cyc)
            plc.write_by_name(plc_loadcyc_name, 0, pyads.PLCTYPE_UINT)
print(load_cyc)

[[5, 5, 5, 5, 5, 5], [5, 5, 5, 5, 5, 5], [5, 5, 5, 5, 5, 5], [5, 5, 5, 5, 5, 5]]
